In [202]:
# модуль библиотеки OpenCV для работы с изображениями
import cv2

# всякие полезные библиотеки для работы с матрицами / отрисовки / etc.
import glob
import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.rcParams['figure.figsize'] = (12, 5)
sns.set_style("white")

import os
from os.path import basename # чтобы достать имя файла из пути


Самолёты я беру из следующих аэропортов:

vko18-gr-small.jpg

jto17-gr1-small.jpg

dme18-gr1-small.jpg.

Т.к. будет аугментация (поворот), то надо брать изображение, где самолёт далеко от краёв, чтобы оно влезло

# 1. Подгружаем данные, смотрим на них

In [203]:
# PATH_TEST = "/srv/samba/share/data/airbus/test/color/1/"
PATH_TEST = "/srv/samba/share/data/airbus/test/validate/mini/"

In [204]:
# Будет лучше, если положительных и отрицательных примеров будет поровну
pos_size = 487
neg_size = 1639 
PATH_POS = "/srv/samba/share/data/airbus/train/16/"
# 11 = 57
# 12 = 30 <---
# 13 = 97
# 14 = 101
# 15 = 720
# 16 = 384
# 3+4c+add1 = 415
# 17 = 487

PATH_NEG = "/srv/samba/share/data/airbus/fail/16+17+/"
# 11 = 40 <---
# 12 = 960
# 13 = 539
# 14 = 4292
# 15 = 462? 476
# 3+4c+add1 = 626
# 17 = 814
# 16+17 = 1458

### обучающая выборка, самолёты

 По идее ОБРЕЗАНИЕ (image = cv2.resize(cv2.imread(name, cv2.IMREAD_GRAYSCALE), (64, 64))) не надо делать, потому что потом надо делать аугментацию. Лучше это сделать после аугментации

In [205]:
def read_faces(path, number):
    """
    Функция для загрузки изображений из папки по пути path.
    """
    names = glob.glob(path + "*.*")
    data = []
    for name in tqdm.tqdm_notebook(names[:number]):
        image = cv2.resize(cv2.imread(name, cv2.IMREAD_GRAYSCALE), (64, 64))
#         image = cv2.imread(name, cv2.IMREAD_GRAYSCALE)
        data.append(image)

    return np.array(data)

In [206]:
data_pos = read_faces(path=PATH_POS, number=pos_size) # получим np.array с N_FACES изображениями лиц размера (64, 64)
labels_pos = np.ones(shape=(len(data_pos), 1)) # np.array c единичками 
print("Read %d images, \ndata_pos shape is %s, \nlabels_pos shape is %s" % (pos_size, data_pos.shape, labels_pos.shape))


Read 487 images, 
data_pos shape is (384, 64, 64), 
labels_pos shape is (384, 1)


### обучающая выборка, фоны

In [207]:
def read_backgrounds(path, number):
    names = glob.glob(path + "*.*")
    data = []
    for name in tqdm.tqdm_notebook(names[:number]):
        image = cv2.resize(cv2.imread(name, cv2.IMREAD_GRAYSCALE), (64, 64))
#         image = cv2.imread(name, cv2.IMREAD_GRAYSCALE)
        data.append(image)
    return np.array(data)

In [208]:
data_neg = read_backgrounds(path=PATH_NEG, number=neg_size)
labels_neg = np.zeros((len(data_neg), 1)) # np.array с нулями
print("Read %d images, \ndata_neg shape is %s, \nlabels_neg shape is %s" % (neg_size, data_neg.shape, labels_neg.shape))

# for one iamge we make (angle_to-angle_from)/angle_step + "1 original"  images
# size of out must be  [(angle_to-angle_from)/angle_step + "1 original" ]* "count of images"
# so [(360-45)/45 + 1] * 40 = 960


Read 1639 images, 
data_neg shape is (1639, 64, 64), 
labels_neg shape is (1639, 1)


### объединяем всё в обучающую выборку

In [209]:
X = np.concatenate([data_pos, data_neg])
y = np.concatenate([labels_pos, labels_neg])
del data_pos, data_neg, labels_pos, labels_neg
print("Shape of X is %s,\nShape of y is %s" % (X.shape, y.shape))

Shape of X is (2023, 64, 64),
Shape of y is (2023, 1)


## 2. Пишем функцию для извлечения HOG

In [210]:
def get_hog_vector(grayscale_image):
    """
    Функция, возвращающая вектор HOG для одного grayscale-изображения размера (64 x 64). 
    Возвращаемый вектор имеет вид (1 x длина вектора HOG).
    """
    
    image_resized = cv2.resize(grayscale_image, (64, 64))
 
    hog_descriptor = cv2.HOGDescriptor((64, 64), #winsize
                                       (32, 32), #blocksize
                                       (16, 16), #blockstride
                                       (8, 8),   #cellsize
                                       9)        #nbins

    return hog_descriptor.compute(image_resized).T

In [211]:
def get_hog_matrix(array_images):
    """
    Функция, возвращающая матрицу, i-я строка которой является вектором HOG для i-го изображения 
    входного массива.
    Возвращаемая матрица имеет вид (число изображений x длина вектора HOG)
    """
    
    # Ваш код здесь
    
#     matrix = np.array([get_hog_vector(image.reshape(128, 128)).flatten() for image in array_images]) 
    matrix = np.array([get_hog_vector(image.reshape(64, 64)).flatten() for image in array_images]) 
    return matrix

In [212]:
X_hog = get_hog_matrix(X)

### 3. Обучаем SVM на признаках HOG нашего датасета

In [213]:
# Отмасштабируем наши признаки, чтобы среднее значение и дисперсия каждого из признаков были равны 0 и 1

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_hog = scaler.fit_transform(X_hog)

In [214]:
# модули для оптимизации моделей машинного обучения
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

In [215]:
X_train, X_test, y_train, y_test = train_test_split(X_hog, y, test_size=0.3)

In [216]:
svm = SVC(C=1e-2, kernel='linear', probability=True)
# svm = SVC(C=1e-1, kernel='sigmoid', probability=True)
svm.fit(X_train, y_train.ravel())

# Ваш код здесь (по желанию).

SVC(C=0.01, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [217]:
y_pred = svm.predict(X_test)

from sklearn.metrics import accuracy_score
print("Accuracy on test set is % .4f" % accuracy_score(y_test, y_pred))
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, digits=5))

Accuracy on test set is  0.9638
             precision    recall  f1-score   support

        0.0    0.97320   0.98129   0.97723       481
        1.0    0.92623   0.89683   0.91129       126

avg / total    0.96345   0.96376   0.96354       607



In [218]:
# Ваш код здесь

info = svm.fit(X_hog, y.ravel())

### 4.1. Пишем функцию для реализации метода sliding window

####  вспомогательные функции

In [219]:
def fileName(path):
    words = re.split('_',basename(path))[:1]
    return '_'.join(words)+'.jpg'

In [220]:
def is_face(image, box):
    y1, x1, y2, x2 = box                      # получаем координаты очередного окна (bbox-а)
    bbox = image[y1:y2, x1:x2]                # "вырезаем" из изображения это окно
    hog = get_hog_vector(bbox)                # подаем на вход функции, вычисляющей HOG
    hog_tr = scaler.transform(hog)            # нормируем вектор
    return svm.predict_proba(hog_tr)[0][1], bbox # с помощью модели предсказываем для окна "вероятность быть самолётом"

### скользящее окно с масштабированием

In [221]:
def sliding_window_multiscale(image, winside=64, step=64, scales=(0.75, 1, 1.25)):
    """
    Функция-генератор для сканирования изображения окном размера (winside x winside) с шагом step.
    Последовательно сканируются изображения в масштабах из scales.
    Возвращает 4-кортеж вида (y1, x1, y2, x2).
    В чем отличие от sliding_window(...)?
    """
    for scale in scales:
        if min(image.shape) < winside / scale:
            continue
            
        for i in range(0, int((image.shape[0] * scale - winside) / step + 1)):
            for j in range(0, int((image.shape[1] * scale - winside) / step + 1)):
                yield (int(i * step / scale), 
                       int(j * step / scale), 
                       int((i * step + winside) / scale), 
                       int((j * step + winside) / scale))

In [222]:
def non_max_suppression_fast(boxes, overlapThresh=0.2):
    if len(boxes) == 0:
        return []
 
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
 
    pick = []
 
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
 
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)
 
    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
 
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])
 
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
 
        overlap = (w * h) / area[idxs[:last]]
 
        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))
 
    return boxes[pick].astype("int")

In [ ]:
def check_on_families_multiscale(scaler, estimator, path, thres=0.95):
    """
    Вспомогательная функция для загрузки и сканирования тестовых изображений и отрисовки найденных лиц на них.
    Изучите работу функции.
    """
    families_list = glob.glob(path + "*.*") # получаем список имен файлов с изображениями
    print(families_list)

    maxv = 255
    for family in families_list:
#     for family in tqdm.tqdm_notebook(families_list):
        image = cv2.imread(family, cv2.IMREAD_GRAYSCALE)
        
        # применяем пороги 175, 200, 235 к изображениям
        ret,thresh1 = cv2.threshold(image,175,maxv,cv2.THRESH_BINARY_INV)
        ret,thresh2 = cv2.threshold(image,200,maxv,cv2.THRESH_BINARY_INV)
        ret,thresh3 = cv2.threshold(image,235,maxv,cv2.THRESH_BINARY_INV)
        
        cv2.imwrite(os.path.splitext(basename(family))[0]+'_othr1.jpg', thresh1)
        cv2.imwrite(os.path.splitext(basename(family))[0]+'_othr2.jpg', thresh2)
#         cv2.imwrite(os.path.splitext(basename(family))[0]+'_othr3.jpg', thresh3)
#         break
    
#         image = cv2.pyrDown(image)                    # теперь в image лежит изображение
        i = 0
        face_boxes = []
        is_face1s = []
        is_face2s = []

        for box in tqdm.tqdm_notebook(sliding_window_multiscale(image, step=8, scales=(0.75, 1, 1.25))):
#         for box in tqdm.tqdm_notebook(sliding_window_multiscale(image, step=8, scales=(1,1) )):
#         for box in sliding_window_multiscale(image, step=8, scales=(0.75, 1, 1.25)):
        
            is_face1, bbox1 = is_face(thresh1, box)
            is_face2, bbox2 = is_face(thresh2, box)
#             is_face3, bbox3 = is_face(thresh3, box)
            is_face3 = 0.0
            
            if (is_face1 > thres) | (is_face2 > thres) | (is_face3 > thres): 
                face_boxes.append(box)                # если вероятность выше порога, сохраняем координаты окна
#                 cv2.imwrite('faces/' + str(i) + '_thr.jpg', image) #пишем окно в файл чтобы потом его hard-negative mining
#                 cv2.imwrite('faces/' + str(i) + '_1.jpg', bbox1) #пишем окно в файл чтобы потом его hard-negative mining
#                 cv2.imwrite('faces/' + str(i) + '_2.jpg', bbox2) #пишем окно в файл чтобы потом его hard-negative mining
#                 cv2.imwrite('faces/' + str(i) + '_3.jpg', bbox3) #пишем окно в файл чтобы потом его hard-negative mining
#                 i = i+1
    
                is_face1s.append(is_face1)
                is_face2s.append(is_face2)

        face_boxes_filtered = non_max_suppression_fast(np.array(face_boxes), overlapThresh=0.2)

        image_draw = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB) # чтобы изображение было цветным
        is_face1s = iter(is_face1s)
        is_face2s = iter(is_face2s)

#         for face_box in tqdm.tqdm_notebook(face_boxes_filtered):
        for face_box in face_boxes_filtered:
            cv2.rectangle(image_draw, (face_box[1], face_box[0]), 
                                      (face_box[3], face_box[2]), 
                                      (0, 0, 255), 1)
            cv2.putText(image_draw, '{:.2f}'.format(next(is_face1s))+','+ '{:.2f}'.format(next(is_face2s)) ,(face_box[1], face_box[0]),cv2.FONT_HERSHEY_SIMPLEX,0.4,(15,255,15))
            #пишем окно в файл чтобы потом его hard-negative mining
#             cv2.imwrite('faces/' + str(i) + '_1.jpg', image[face_box[1]:face_box[0],face_box[3]:face_box[2]]) 
#             cv2.imwrite('faces/' + str(i) + '_1.jpg', thresh1[face_box[0]:face_box[2],face_box[1]:face_box[3]]) 
#             cv2.imwrite('faces/' + str(i) + '_2.jpg', thresh1[face_box[0]:face_box[2],face_box[1]:face_box[3]]) 
            i = i+1

        image_draw = cv2.putText(image_draw,'Image=' + family.split('/')[-1],(10,35),cv2.FONT_HERSHEY_SIMPLEX,1,(255,15,15))
#         image_draw = cv2.putText(image_draw,'DIR_POS='+PATH_POS.split('/')[-2],(10,70),cv2.FONT_HERSHEY_SIMPLEX,1,(255,15,15))
#         image_draw = cv2.putText(image_draw,'DIR_NEG='+PATH_NEG.split('/')[-2],(10,105),cv2.FONT_HERSHEY_SIMPLEX,1,(255,15,15))
#         image_draw = cv2.putText(image_draw,'pos_size='+str(pos_size),(10,140),cv2.FONT_HERSHEY_SIMPLEX,1,(255,15,15))
#         image_draw = cv2.putText(image_draw,'neg_size='+str(neg_size),(10,175),cv2.FONT_HERSHEY_SIMPLEX,1,(255,15,15))
#         image_draw = cv2.putText(image_draw,'svminfo='+repr(info),(10,210),cv2.FONT_HERSHEY_SIMPLEX,1,(255,15,15))
#         image_draw = cv2.putText(image_draw,'thr='+str(thres),(10,245),cv2.FONT_HERSHEY_SIMPLEX,1,(255,15,15))
        cv2.imwrite(os.path.splitext(basename(family))[0]+'_out.jpg', image_draw)

In [ ]:
# plt.rcParams['figure.figsize'] = (30, 30)
%time check_on_families_multiscale(scaler, svm, path=PATH_TEST, thres=0.75)

['/srv/samba/share/data/airbus/test/validate/mini/lga18-1-mini1.jpg', '/srv/samba/share/data/airbus/test/validate/mini/led18-mini.jpg', '/srv/samba/share/data/airbus/test/validate/mini/sip18-mini.jpg', '/srv/samba/share/data/airbus/test/validate/mini/lga18-1-mini2.jpg']


# Идеи на потом

1. Реализовать отдельный классификатор по уровням фильтрации порога. А может и не нужно, потому что 
2. Реализовать двойной фильтр как в Гимпе. Уж очень он хороший результат дал для первой выборки